In [14]:
"""

@author: bkearney
email: bkearne5@uncc.edu

Purpose: improve from TranslationfromCSV code with new datasets and more ubiquitous code

"""

import numpy as np
import pandas as pd
import re
import os
import statistics 
# import allel

# Change directory to path with all reference/input files
%cd C:\Users\Brendan\Documents\Summer20

gene = pd.read_table('gene2ensembl')
human = pd.read_table('Homo_sapiens.gene_info')
mart = pd.read_csv('mart_export.txt',delimiter=',')

# Create Aliases column (delimited Synonyms)
aliases = human['Synonyms'].tolist()
aliases_sep = []
for i in range(len(aliases)):
    my_list = aliases[i].split("|")
    my_list.append(human['Symbol'].iloc[i])
    aliases_sep.append(my_list)
human['Aliases'] = aliases_sep

# Create geneId column from splicing/filtering dbXrefs
dbXrefs = human['dbXrefs'].tolist()
geneIds = []
for i in range(len(dbXrefs)):
    if 'Ensembl' in dbXrefs[i]:
        gene_id = dbXrefs[i].split("Ensembl:")[1]
    else:
        gene_id = ""
    geneIds.append(gene_id)
human['geneId'] = geneIds

[Errno 2] No such file or directory: 'C:UsersBrendanDocumentsSummer20'
/Users/bkearney/Summer_20


/Users/bkearney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/Users/bkearney/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type,Aliases,geneId
22260,9606,353500,BMP8A,-,OP-2,HGNC:HGNC:21650|Ensembl:ENSG00000183682,1,1p34.3,bone morphogenetic protein 8a,protein-coding,BMP8A,bone morphogenetic protein 8a,O,bone morphogenetic protein 8A|BMP-8A,20200313,-,"[OP-2, BMP8A]",ENSG00000183682


In [5]:
#old

def translateList(list_features, testing_df):
    
    test_col = list_features[0]
    reference_df = list_features[1]
    reference_col = list_features[2]
    output_col = list_features[3]

    start_param = testing_df[test_col].tolist()
    in_list = reference_df[reference_col].tolist()

    out_listoflist = []

    for i in range(len(start_param)):
        param = start_param[i]

        match_list=[]
        for j in range(len(in_list)):
            if str(param) in str(in_list[j]):
                index_match = reference_df.iloc[j]
                match_list.append(index_match)

        out_list = []
        for k in range(len(match_list)):
            out_list.append(match_list[k][output_col])

        out_listoflist.append(out_list)
    testing_df['output '+reference_col]=out_listoflist

    return testing_df
#     'geneId (GRCH37.66)','human','geneId','Full_name_from_nomenclature_authority'

def decimal_removal(database, col_index):  # make bp locs into whole numbers for translation
    temp_list = database.iloc[:,col_index].tolist()
    end_list = []
    
    for value in temp_list:
        end_list.append(str(value).split('.',1)[0])
        
    return end_list
    print(end_list)

In [6]:
def f_score(TP,FP,FN):
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f_dia = 2 * precision * recall / (precision+recall)
    return f_dia

In [10]:
#old

def scoring(dataF, colname):

    nonmatches_ind = [ind for ind, x in enumerate(dataF[colname]) if len(x)==0 or x != x or x=='-']
    matches_ind = [ind for ind, x in enumerate(dataF[colname]) if len(x)!=0 or x == x or x!='-']
    nonmatches = dataF.iloc[nonmatches_ind]
    matches = dataF.iloc[matches_ind]

 

    truePos = len(dataF)-len(nonmatches)
    falsePos = 0 # How to calculate FP?
    falseNeg = 0
    fScore = f_score(truePos,falsePos,falseNeg)
#     return fScore
    print(len(nonmatches))
    print(len(dataF))
    return round(100*len(nonmatches)/len(dataF),4)


In [89]:
#current function
def combined_translate(input_file, file_type, trans_list):
    if (file_type == 'csv'):
        df = pd.read_csv(input_file)
    if (file_type == 'excel'):
        df = pd.read_excel(input_file)
    if (file_type == 'other'):
        df = pd.read_table(input_file)
    
    for item in trans_list:
        test_col = item[0]
        reference_df = item[1]
        reference_col = item[2]
        output_col = item[3]
        
        start_param = df[test_col].tolist()
        in_list = reference_df[reference_col].tolist()
        out_listoflist = []

        for i in range(len(start_param)):
            param = start_param[i]

            match_list=[]
            for j in range(len(in_list)):
                if str(param) in str(in_list[j]):
                    index_match = reference_df.iloc[j]
                    match_list.append(index_match)

            out_list = []
            for k in range(len(match_list)):
                out_list.append(match_list[k][output_col])

            out_listoflist.append(out_list)
        out_colname = 'output ' + reference_col
        df[out_colname] = out_listoflist

    
        nonmatches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)==0 or x != x or x=='-']
        matches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)!=0 or x == x or x!='-']
        nonmatches = df.iloc[nonmatches_ind]
        matches = df.iloc[matches_ind]
        print(round(100*len(nonmatches)/len(df),4)," percent mismatch, ",test_col)
    
    print(df)
    
        
          

In [31]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4460634/
# Supplementary materials, 
# GPEA analysis of the RNAseq UC GRN for gene families

gpea = pd.read_csv('UC_GPEA.csv')
gpea_list = ['tag',human,'Aliases','Full_name_from_nomenclature_authority']
gpea_translated = translateList(gpea_list, gpea)
score_gpea = 100-scoring(gpea_translated, 'output Aliases')
# gpea_translated['output Aliases']
score_gpea


18
103


82.5243

In [32]:
""" combined_translate() template
copy this template and fill with parameters

*******   COPY   ********

### Name of input data file
input_file = ''

### Type of file (excel, csv, or other)
file_type = ''

### translation 1 test col name
test_col1 = ''

### translation 1 reference dataset (human,gene,mart)
ref_df1 =

### translation 1 reference col name
ref_col1 = ''
    
### translation 1 output col name
out_col1 = ''

### translation 1 use only if one translation, otherwise delete
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]

### translation 2 test col name (delete all until end)
test_col2 = ''

### translation 2 reference dataset (human,gene,mart)
ref_df2 = 

### translation 2 reference col name
ref_col2 = ''

### translation 2 output col name
out_col2 = ''

### use for two translations
trans_list = [[test_col1,ref_df1,ref_col1,out_col1],[test_col2,ref_df2,ref_col2,out_col2]]


combined_translate(input_file,file_type,trans_list)

************************

"""


' combined_translate() template\ncopy this template and fill with parameters\n*******   COPY   ********\n\n\n\n'

In [67]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3748065/
# Table 2

### Name of input data file
input_file = 'TCGA.rtf'
# input_file = 'UC_GPEA.csv'
### Type of file (excel, csv, or other)
file_type = 'csv'

### translation 1 test col name
test_col1 = 'gene'

### translation 1 reference dataset (human,gene,mart)
ref_df1 = human

### translation 1 reference col name
ref_col1 = 'Aliases'
    
### translation 1 output col name
out_col1 = 'Full_name_from_nomenclature_authority'

### translation 1 use only if one translation, otherwise delete
trans_list1 = [[test_col1,ref_df1,ref_col1,out_col1]]


combined_translate(input_file,file_type,trans_list)

        gene                                     output Aliases
0      BMP8A                    [bone morphogenetic protein 8a]
1   CEACAM20                    [CEA cell adhesion molecule 20]
2       CHGA                                   [chromogranin A]
3     COL9A1  [collagen type IX alpha 1 chain, collagen type...
4        DCD  [desmin, isocitrate dehydrogenase (NADP(+)) 1,...
5      GPM6A                                 [glycoprotein M6A]
6      GRIA3  [glutamate ionotropic receptor AMPA type subun...
7      GSTA3                [glutathione S-transferase alpha 3]
8       IGF2  [insulin like growth factor 2, insulin like gr...
9      KLK11                  [kallikrein related peptidase 11]
10     KRT16  [keratin 16, keratin 16 pseudogene 4, keratin ...
11     KRT6A                                       [keratin 6A]
12     LEMD1  [LEM domain containing 1, LEMD1 antisense RNA ...
13    LGALS7                          [galectin 7, galectin 7B]
14      LY6D  [glycosylphosphatidylinosi

In [92]:
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4460634/
# Supplementary materials, 
# GPEA analysis of the RNAseq UC GRN for gene families

### Name of input data file
input_file = 'UC_GPEA.csv'

### Type of file (excel, csv, or other)
file_type = 'csv'

### translation 1 test col name
test_col1 = 'tag'

### translation 1 reference dataset (human,gene,mart)
ref_df1 = human

### translation 1 reference col name
ref_col1 = 'Aliases'
    
### translation 1 output col name
out_col1 = 'Full_name_from_nomenclature_authority'

### translation 1 use only if one translation, otherwise delete
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]

combined_translate(input_file,file_type,trans_list)


17.4757  percent mismatch,  tag
          tag                                               name  \
0          CD                                       CD molecules   
1        HIST                   Histones / Replication-dependent   
2       KRTAP                        Keratin associated proteins   
3       SNORA  ncRNAs / Small nucleolar RNAs : H/ACA box cont...   
4       ZKRAB                                                  -   
5         ZNF                             Zinc fingers C2H2-type   
6       PCDHC             Cadherins / Protocadherins : Clustered   
7         RPL                               L ribosomal proteins   
8         KRT                                                  -   
9         HLA                         Histocompatibility complex   
10        IGD  Immunoglobulin superfamily / Immunoglobulin-li...   
11     LNCRNA                               Long non-coding RNAs   
12      C1SET  Immunoglobulin superfamily / C1-set domain con...   
13       HOXL   

In [93]:
# Drug study from google docs file

### Name of input data file
input_file = 'drug_supplement.xlsx'

### Type of file (excel, csv, or other)
file_type = 'excel'

### translation 1 test col name
test_col1 = 'Gene'

### translation 1 reference dataset (human,gene,mart)
ref_df1 = human

### translation 1 reference col name
ref_col1 = 'Aliases'
    
### translation 1 output col name
out_col1 = 'description'

### translation 2 test col name (delete all until end)
test_col2 = 'Protein'

### translation 2 reference dataset (human,gene,mart)
ref_df2 = human

### translation 2 reference col name
ref_col2 = 'description'

### translation 2 output col name
out_col2 = 'Symbol'

### use for two translations
trans_list = [[test_col1,ref_df1,ref_col1,out_col1],[test_col2,ref_df2,ref_col2,out_col2]]


combined_translate(input_file,file_type,trans_list)

10.7143  percent mismatch,  Gene
100.0  percent mismatch,  Protein
            Gene                                        Protein  \
0          OPRM1                              μ-opioid receptor   
1          OPRK1                              κ-opioid receptor   
2           PDYN                                Preprodynorphin   
3             TH                           Tyrosine hydroxylase   
4           DRD2                           Dopamine receptor D2   
5           DRD3                           Dopamine receptor D3   
6           DRD4                           Dopamine receptor D4   
7            DBH                        Dopamine β-hydroxylase    
8   DAT (SLC6A3)                           Dopamine transporter   
9           TPH1                       Tryptophan hydroxylase 1   
10          TPH2                       Tryptophan hydroxylase 2   
11         HTR1B                          Serotonin receptor 1B   
12         HTR2A                          Serotonin receptor 2